In [1]:
import pandas as pd
# features = pd.read_excel("LteFeatures1.xlsx")

In [2]:
params = pd.read_excel("LteParams.xlsx")

In [3]:
import pymongo
# retrieve all documents from the collection
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["rasa"]
collection = db["Chap2"]
data = list(collection.find())
# create a Pandas DataFrame from the list of dictionaries
Xdf = pd.DataFrame(data)

In [11]:
# # Xdf
# # create a new row
# new_row = {'category': 4, 'col2': 'd'}

# # add the new row to the end of the dataframe
# Xdf.loc[len(Xdf)] = new_row
# new_col=[30,30,20,20,20,30,20,20,20,20]
new_col=[20,20,20,20,20,20,20,20,20,20]
df["pmHoPrepSuccLTEInterF"]=new_col

In [4]:
Xdf = Xdf.drop('_id', axis=1)
paramlist = Xdf.values.tolist()
solutionslist=[]
next=[]
collection = db['Conteurs']
# retrieve documents with specified columns
documents = collection.find({})
df = pd.DataFrame(list(documents))
k=0
j=0

In [82]:
Xdf

,category,next,solution,counter_value,counters
0,IRAT Handove,"Handover Preparation Failure, Handover Executi...",not ready,0,none
1,Handover Preparation Failure,"Target Cell Availibility, configuration issue,...",not ready,(pmHoPrepAtt-pmHoPrepSucc)<pmHoExeSucc,"[pmHoPrepAtt, pmHoPrepSucc]"
2,Handover Execution Failure,"Poor Coverage,Target cell High UL interference...",not ready,(pmHoExeAtt-pmHoExeSucc)<pmHoExeSucc,"[pmHoExeAtt, pmHoExeSucc]"
3,Target Cell Availibility,none,target site is disabled / sector down / softwa...,0,none
4,configuration issue,none,"Frequency relation (UtranFreqRelation) , neigh...",0,none
5,HW issues,none,"Check HW issues , alarms , improper cabling , ...",0,none
6,MCPC,MCPCparams,not ready,0,none
7,MCPCparams,MCPCSolutions,not ready,pmCriticalBorderEvalReport>pmBadCovSearchEvalR...,"[pmCriticalBorderEvalReport, pmBadCovSearchEva..."
8,MCPCSolutions,none,Tune parameters according to number of carrier...,0,none
9,LTE Intra/Inter Frequency(HOSR),"LTE Handover Preparation Failure, LTE Handover...",not ready,0,none


In [83]:

for param in paramlist:
  param[1]=param[1].replace(' ', '')
  param[0]=param[0].replace(' ', '')
  if param[0] in next:
    if param[4]== 'none' and param[2]!= 'not ready':
       solutionslist.append(param[4])
    if param[2]=='not ready' and param[4]!='none':
        for x in param[4]:
           if x.replace(" ", "") not in df.columns:
              print(x + " values not added in database")
           else: k = 1
        if k==1:
           i=param[4]
           if len(i)== 2:
              j+=1
              i[0].replace(" ", "") 
              i[1].replace(" ", "")
              if i[0] in df.columns:
               result = df.apply(lambda row: eval(param[3], {i[0].replace(" ", ""): row[i[0].replace(" ", "")], i[1].replace(" ", ""): row[i[1].replace(" ", "")]}), axis=1) 
              df["resultat "+param[0]]=result
  if param[2]=='notready':
    next=param[1].split(",")
   
  

In [84]:
# get the names of columns that contain 'resultat'
s=[]
for j in range(len(df)):
   s.append('')
s

['', '', '', '', '', '', '', '', '', '']

In [85]:

sol=""
solist=[]
#Determine a solution to the problems
next=""   
for i in range(len(df)):
   for col in df.columns:
      if df[col][i]==False:
        if "resultat" in col:
            solution=[]
            solist=[] 
            x=col.replace("resultat ","")
            my_string=''
            for param in paramlist:
               if param[0]==x:
                  next=param[1]
               if next!="" :
                  for y in next.split(","):
                     if y==param[0]:
                        if param[2] not in solution:
                           solution.append(param[2])              
            my_string = ' '.join(solution)
            my_string='for '+x+' : '+my_string 
            if my_string not in solution:
               solist.append(my_string)
            sol=''.join(solist)
            # print(sol)
            if sol not in s[i]:
               s[i]=s[i]+sol+' ;\n '
               sol=''
            print(i)
            print(s[i])
            next=""
                     


0
for MCPCparams : TuneparametersaccordingtonumberofcarriersinLTE ;
 
0
for MCPCparams : TuneparametersaccordingtonumberofcarriersinLTE ;
 for HandoverPreparationFailure : targetsiteisdisabled/sectordown/softwarepackagecorrupt Frequencyrelation(UtranFreqRelation),neighborrelation(ExternalUtrancell) CheckHWissues,alarms,impropercabling,connectorissues,VSWR ;
 
0
for MCPCparams : TuneparametersaccordingtonumberofcarriersinLTE ;
 for HandoverPreparationFailure : targetsiteisdisabled/sectordown/softwarepackagecorrupt Frequencyrelation(UtranFreqRelation),neighborrelation(ExternalUtrancell) CheckHWissues,alarms,impropercabling,connectorissues,VSWR ;
 for Neighborcellloadissue : Checkthesecountersontargetcell,Reduceno.ofHObyphysicaloptiorfeatures(UELevelOscillatingHandoverMinimization&AutomatedMobilityOptimization),MPcapacityofDUSishigherthanDUL,toincreaseMPcapacityDULcanbereplacedbyDUSPhysicalchanges(Tilt,Azimuth,heightofantenna)tooffloadtrafficfromtargetcellOffloadTargetcellLoadbychangingLa

In [74]:
df['solution']=s
# s[9]

In [75]:
df

,_id,OSS_ID,SN,ELEMENT,MOID,pmHoExeSucc,pmHoExeAtt,pmCriticalBorderEvalReport,pmBadCovSearchEvalReport,resultat HandoverExecutionFailure,resultat MCPCparams,pmHoPrepAttLTEIntraF,pmHoPrepSuccLTEIntraF,pmHoPrepAttLTEInterF,pmHoPrepSuccLTEInterF,resultat HandoverPreparationFailure,resultat Neighborcellloadissue,solution
0,64186345be2bea336c2ad68c,eniq_oss_1,"SubNetwork=ONRM_ROOT_MO,SubNetwork=SRAN,SubNet...",B4G3G_9_AVRIL,"ManagedElement=B4G3G_9_AVRIL,ENodeBFunction=1,...",30,40,30,40,True,False,30,30,30,20,False,False,for MCPCparams : Tuneparametersaccordingtonumb...
1,64186345be2bea336c2ad68d,eniq_oss_1,"SubNetwork=ONRM_ROOT_MO,SubNetwork=SRAN,SubNet...",B4G3G_9_AVRIL,"ManagedElement=B4G3G_9_AVRIL,ENodeBFunction=1,...",30,40,30,10,True,True,20,30,30,20,True,True,
2,64186345be2bea336c2ad68e,eniq_oss_1,"SubNetwork=ONRM_ROOT_MO,SubNetwork=SRAN,SubNet...",B4G3G_9_AVRIL,"ManagedElement=B4G3G_9_AVRIL,ENodeBFunction=1,...",30,40,30,10,True,True,20,20,20,20,True,True,
3,64186345be2bea336c2ad68f,eniq_oss_1,"SubNetwork=ONRM_ROOT_MO,SubNetwork=SRAN,SubNet...",B4G3G_9_AVRIL,"ManagedElement=B4G3G_9_AVRIL,ENodeBFunction=1,...",30,40,30,10,True,True,20,20,20,20,True,True,
4,64186345be2bea336c2ad690,eniq_oss_1,"SubNetwork=ONRM_ROOT_MO,SubNetwork=SRAN,SubNet...",B4G3G_9_AVRIL,"ManagedElement=B4G3G_9_AVRIL,ENodeBFunction=1,...",30,40,30,10,True,True,20,20,20,20,True,True,
5,64186345be2bea336c2ad691,eniq_oss_1,"SubNetwork=ONRM_ROOT_MO,SubNetwork=SRAN,SubNet...",B4G3G_9_AVRIL,"ManagedElement=B4G3G_9_AVRIL,ENodeBFunction=1,...",30,70,30,20,False,True,30,30,30,20,True,True,"for HandoverExecutionFailure : CheckHWissues,a..."
6,64186345be2bea336c2ad692,eniq_oss_1,"SubNetwork=ONRM_ROOT_MO,SubNetwork=SRAN,SubNet...",B4G3G_AeroportGabes,"ManagedElement=B4G3G_AeroportGabes,ENodeBFunct...",30,70,30,20,False,True,20,20,20,20,True,True,"for HandoverExecutionFailure : CheckHWissues,a..."
7,64186345be2bea336c2ad693,eniq_oss_1,"SubNetwork=ONRM_ROOT_MO,SubNetwork=SRAN,SubNet...",B4G3G_AeroportGabes,"ManagedElement=B4G3G_AeroportGabes,ENodeBFunct...",30,70,30,20,False,True,20,20,20,20,True,True,"for HandoverExecutionFailure : CheckHWissues,a..."
8,64186345be2bea336c2ad694,eniq_oss_1,"SubNetwork=ONRM_ROOT_MO,SubNetwork=SRAN,SubNet...",B4G3G_AeroportGabes,"ManagedElement=B4G3G_AeroportGabes,ENodeBFunct...",30,70,30,20,False,True,20,20,20,20,True,True,"for HandoverExecutionFailure : CheckHWissues,a..."
9,64186345be2bea336c2ad695,eniq_oss_1,"SubNetwork=ONRM_ROOT_MO,SubNetwork=SRAN,SubNet...",B4G3G_AeroportGabes,"ManagedElement=B4G3G_AeroportGabes,ENodeBFunct...",30,70,30,40,False,False,20,20,20,20,False,False,"for HandoverExecutionFailure : CheckHWissues,a..."


In [76]:
dfknn=pd.concat([df.iloc[:, 9:11], df.iloc[:, 16:18]], axis=1)
# Convert boolean columns to numeric (0 or 1)
dfknn['resultat HandoverExecutionFailure'] = dfknn['resultat HandoverExecutionFailure'].astype(int)
dfknn['resultat MCPCparams'] = dfknn['resultat MCPCparams'].astype(int)
dfknn['resultat Neighborcellloadissue'] = dfknn['resultat Neighborcellloadissue'].astype(int)
# Repeat each row 3 times
dfknn = dfknn.loc[df.index.repeat(4)].reset_index(drop=True)
solution=dfknn['solution']
dfknn= dfknn.drop('solution', axis=1)
dfknn

,resultat HandoverExecutionFailure,resultat MCPCparams,resultat Neighborcellloadissue
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,1,1
5,1,1,1
6,1,1,1
7,1,1,1
8,1,1,1
9,1,1,1


# Using knn on the counters dataframe

In [77]:
# We used pmHoExeSucc, pmHoExeAtt, pmCriticalBorderEvalReport, and pmBadCovSearchEvalReport as features. using iloc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dfknn,solution, test_size=0.4, random_state=43)

# Scale the data using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn.predict(X_test)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 1.0


In [78]:
print(X_test)

    resultat HandoverExecutionFailure  resultat MCPCparams  \
19                                  1                    1   
13                                  1                    1   
24                                  0                    1   
31                                  0                    1   
7                                   1                    1   
6                                   1                    1   
35                                  0                    1   
9                                   1                    1   
20                                  0                    1   
37                                  0                    0   
10                                  1                    1   
32                                  0                    1   
14                                  1                    1   
8                                   1                    1   
12                                  1                    1   
5       

In [79]:
print(y_test[37])

for HandoverExecutionFailure : CheckHWissues,alarms,impropercabling,connectorissues,VSWR ;
 for MCPCparams : TuneparametersaccordingtonumberofcarriersinLTE ;
 for HandoverPreparationFailure : targetsiteisdisabled/sectordown/softwarepackagecorrupt Frequencyrelation(UtranFreqRelation),neighborrelation(ExternalUtrancell) CheckHWissues,alarms,impropercabling,connectorissues,VSWR ;
 for Neighborcellloadissue : Checkthesecountersontargetcell,Reduceno.ofHObyphysicaloptiorfeatures(UELevelOscillatingHandoverMinimization&AutomatedMobilityOptimization),MPcapacityofDUSishigherthanDUL,toincreaseMPcapacityDULcanbereplacedbyDUSPhysicalchanges(Tilt,Azimuth,heightofantenna)tooffloadtrafficfromtargetcellOffloadTargetcellLoadbychangingLayeringstrategy,Idlemodecellreselection,connectedmodemobility&Loadbalancingparameters ;
 


In [80]:
print(y_pred)

['' ''
 'for HandoverExecutionFailure : CheckHWissues,alarms,impropercabling,connectorissues,VSWR ;\n '
 'for HandoverExecutionFailure : CheckHWissues,alarms,impropercabling,connectorissues,VSWR ;\n '
 '' ''
 'for HandoverExecutionFailure : CheckHWissues,alarms,impropercabling,connectorissues,VSWR ;\n '
 ''
 'for HandoverExecutionFailure : CheckHWissues,alarms,impropercabling,connectorissues,VSWR ;\n '
 'for HandoverExecutionFailure : CheckHWissues,alarms,impropercabling,connectorissues,VSWR ;\n for MCPCparams : TuneparametersaccordingtonumberofcarriersinLTE ;\n for HandoverPreparationFailure : targetsiteisdisabled/sectordown/softwarepackagecorrupt Frequencyrelation(UtranFreqRelation),neighborrelation(ExternalUtrancell) CheckHWissues,alarms,impropercabling,connectorissues,VSWR ;\n for Neighborcellloadissue : Checkthesecountersontargetcell,Reduceno.ofHObyphysicaloptiorfeatures(UELevelOscillatingHandoverMinimization&AutomatedMobilityOptimization),MPcapacityofDUSishigherthanDUL,toincrease